In [ ]:
import requests

response = requests.get('http://127.0.0.1:5000/hello')
print(response.text)

data = {'a': 1, 'b': 2, 'c' : 3}
response = requests.post('http://127.0.0.1:5000/api', json=data)
print(response.json())